### Questions
* recursive functions
    * how 
    * when
* kotlin

### Objectives
* YWBAT parse a json file effectively

### Outline
* Questions
* loading in jsons from mongo
* using mongo db to access data and find insights on text data

### What does noSQL mean?
* no relational tables
* in no sql, we lose structure and therefore lose joins


### What is a json file
* it's basically a dictionary, with some minor restraints

# Loading in string as dict

In [27]:
stringjson = '{"key":"value"}'
stringjson

'{"key":"value"}'

In [28]:
stringd = json.loads(stringjson)
stringd

{'key': 'value'}

# Loading json file as dict

In [8]:
import json

In [22]:
with open("/Users/rafael/file.json") as f:
    d = json.load(f)
d

{'key': 'value',
 'key2': 'value2',
 'students': [{'age': 28, 'name': 'Bryan DiCarlo', 'state': 'TX'},
  {'age': 21, 'name': "Jonathan Ericksen's", 'state': 'not Tx'}]}

### Installing Mongo on a Mac
* install homebrew
* brew install mongo
 
* collections = tables
* documents = rows

In [72]:
import json
import pandas as pd
import numpy as np

from pprint import pprint

from textblob import TextBlob
from pymongo import MongoClient
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
client = MongoClient(host='localhost', port=27017)

In [31]:
client.list_database_names()

['admin',
 'config',
 'local',
 'marchmadness',
 'music_tweets',
 'mydb',
 'new_db',
 'tweets']

In [37]:
musicdb = client["music_tweets"]

In [40]:
kendricktweets = musicdb["kendrickLamar"]

In [93]:
dlist = []
for index, tweet in enumerate(kendricktweets.find({})): # select * from kendrickLamar;
    d = dict()
    d['text'] = tweet["text"]
    blob = TextBlob(text)
    d['polarity'] = blob.sentiment.polarity
    d['subjectivity'] = blob.sentiment.subjectivity
    
    source = tweet["source"]
    soup = BeautifulSoup(source, 'html.parser')
    res = soup.find('a')
    d["source"] = res.contents[0]
    
    d['coor'] = tweet['coordinates']
    d['location'] = tweet['user']['location']
    if index%200 == 0:
        print("finished {} tweets".format(index))
    dlist.append(d)

finished 0 tweets
finished 200 tweets
finished 400 tweets
finished 600 tweets
finished 800 tweets
finished 1000 tweets
finished 1200 tweets
finished 1400 tweets
finished 1600 tweets


In [94]:
df = pd.DataFrame(dlist)
df.head()

coor       location  polarity                source  subjectivity  \
0  None   Indiana, USA       0.0  Radio.co now playing           0.0   
1  None           Why?       0.0    Twitter for iPhone           0.0   
2  None           None       0.0    Twitter for iPhone           0.0   
3  None  Nightosphere.       0.0    Twitter for iPhone           0.0   
4  None   Jeno's smile       0.0   Twitter for Android           0.0   

                                                text  
0  Now Playing: Tech N9ne f. Kendall Morgan, Kend...  
1  RT @Bj532x: امبي وايد حلوه 💔 https://t.co/2oDR...  
2  @JayStephMD @kendricklamar Bitch can we win a ...  
3  Well this is trash cause 4:44 &amp; u missed W...  
4  RT @archivelyric: pray for me - the weeknd ft....

In [95]:
df.location.value_counts()

New York, NY                     125
Johannesburg, South Africa        21
South Africa                      19
Florida, USA                      18
Nairobi, Kenya                    17
Lagos, Nigeria                    16
United States                     16
Los Angeles, CA                   15
Accra, Ghana                      11
Paris, France                     11
London, England                    8
Houston, TX                        8
Worldwide                          7
Cork, Ireland                      6
California, USA                    5
Paris                              5
Port Elizabeth, South Africa       5
Chicago, IL                        5
Michigan, USA                      5
Nigeria                            5
France                             5
Nashville, TN                      5
Miami, FL                          5
Boston, MA                         4
Dubai, United Arab Emirates        4
United Kingdom                     4
Warri                              4
K

In [65]:
from string import ascii_letters
ascii_letters

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [67]:
all(ch in ascii_letters for ch in df.text[0])

False

In [88]:
for index, tweet in enumerate(kendricktweets.find({}, limit=5)): # select * from kendrickLamar;
    pprint(tweet)
    break

{'_id': ObjectId('5c55441d26596444e00f9b3b'),
 'contributors': None,
 'coordinates': None,
 'created_at': 'Sat Feb 02 07:17:43 +0000 2019',
 'entities': {'hashtags': [{'indices': [108, 116], 'text': 'GetLive'},
                           {'indices': [118, 138],
                            'text': 'GetLiveMediaNetwork'}],
              'symbols': [],
              'urls': [],
              'user_mentions': [{'id': 776128881107558401,
                                 'id_str': '776128881107558401',
                                 'indices': [97, 107],
                                 'name': '7Six5Live Radio',
                                 'screen_name': '7Six5Live'}]},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': None,
 'id': 1091596511681630208,
 'id_str': '1091596511681630208',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote

### Assessment